# CUDA编程模型--- 执行流 和 运行库

#### 1.CUDA流
CUDA程序的并行层次主要有两个，一个是核函数内部的并行，一个是核函数的外部的并行。我们之前讨论的都是核函数的内部的并行。核函数外部的并行主要指：
- 核函数计算与数据传输之间的并行
- 主机计算与数据传输之间的并行
- 不同的数据传输之间的并行
- 核函数计算与主机计算之间的并行
- 不同核函数之间的并行


CUDA流表示一个GPU操作队列，该队列中的操作将以添加到流中的先后顺序而依次执行。我们的所有CUDA操作都是在流中进行的，虽然我们可能没发现，但是有我们前面的例子中的指令，内核启动，都是在CUDA流中进行的，只是这种操作是隐式的，所以肯定还有显式的，所以，流分为：
- 隐式声明的流，我们叫做空流
- 显式声明的流，我们叫做非空流

基于流的异步内核启动和数据传输支持以下类型的粗粒度并发：
- 重叠主机和设备计算
- 重叠主机计算和主机设备数据传输
- 重叠主机设备数据传输和设备计算
- 并发设备计算（多个设备）

接下来，我们就完成下面这个核函数，在两个流并发的实现：
```C++
__global__ void kernel( int *a, int *b, int *c ) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < N) {
        int idx1 = (idx + 1) % 256;
        int idx2 = (idx + 2) % 256;
        float   as = (a[idx] + a[idx1] + a[idx2]) / 3.0f;
        float   bs = (b[idx] + b[idx1] + b[idx2]) / 3.0f;
        c[idx] = (as + bs) / 2;
    }
}
```

创建[stream.cu](stream.cu)文件，详情请参考[result1.cu](result1-stream.cu)

修改Makefile，利用Makefile编译，并执行

In [ ]:
!make

In [ ]:
!./stream

利用nvprof测试程序性能

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./stream

删除其中一个流，并测试性能，如果遇到麻烦，请参考[result2.cu](result2.cu)

利用Makefile文件编译，并执行程序

In [ ]:
!make

In [ ]:
!./stream

#### 2.cuBLAS  
cuBLAS 库是基于 NVIDIA®CUDA™ 运行时的 BLAS（基本线性代数子程序）的实现。它允许用户访问 NVIDIA GPU 的计算资源。


在[cublas_gemm.cu](cublas_gemm.cu)中使用```cublasDgemm()```函数，如果遇到麻烦，请参考[result3.cu](result3.cu)

修改Makefile文件，这里需要将```$(CC)  $(TEST_SOURCE) -o  $(TARGETBIN)``` 修改为```$(CC)  $(TEST_SOURCE) -lcublas -o  $(TARGETBIN)```  

编译，并执行程序

In [ ]:
!make

In [ ]:
!./cublas_gemm

利用nvprof查看程序性能

In [ ]:
!sudo /usr/local/cuda/bin/nvprof ./cublas_gemm

课后作业：
1. 尝试调用cublas做矩阵乘法和向量加法操作，跟之前自己写的程序对比，查看性能差距，并分析可能改进的地方？
2. 如果本地文件存储着2个1000000*1000000的矩阵，我们想将这两个矩阵进行乘积，如何操作？